In [1]:
from llama_index.embeddings.ollama import OllamaEmbedding
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_parse import LlamaParse
from llama_index.llms.ollama import Ollama
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.readers.base import BaseReader
from llama_index.core import Document
import pymupdf4llm
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import VectorStoreIndex
import nest_asyncio
# load it again to confirm it worked
from llama_index.core import StorageContext, load_index_from_storage

In [5]:
index = load_index_from_storage(StorageContext.from_defaults(persist_dir="/workspace/storage"), index_id="text")
ollama_base_url=os.environ['OLLAMA_BASE_URL']
ollama = Ollama(model="llama3.1:8b", request_timeout=120.0, base_url=ollama_base_url)
ollama_embeding = OllamaEmbedding(model_name="nomic-embed-text", request_timeout=120.0, base_url=ollama_base_url, ollama_additional_kwargs={"mirostat": 0},)
Settings.llm = ollama
Settings.embed_model = ollama_embeding


In [6]:
retriever = index.as_retriever(verbose=True)
response = retriever.retrieve("new neighbor behaved churlishly ")
for res in response:
    print(res.text)

![](pdf_images/Oleg 11_30.pdf-3-0.png)

1. Contrite

1. Contrite

2. Succulent

2. Succulent - The Thanksgiving dinner started with a succulent turkey that

3. Complacent

the family cooked every year.

4. Churlish

3. Complacent - After many months in the gym, he felt complacent and

5. Buffet (v)

stopped pushing hard. 6. Buffet (n)

4. Churlish - The new neighbor behaved churlishly at the beginning but, with 7. Bemuse

8. Buffoon

time, became friendlier.

5. Buffet (v) - He was driving on the highway when something started
buffeting on the back of his car.

6. Buffet (n) - The celebration finished with a big party accompanied by a
delicious buffet.

7. Bemuse - The presenter entered the wrong room, which bemused the
listeners.

8. Buffoon - Behaving like a buffoon, a drunk neighbor ruined the party,
which made all guests leave earlier.


-----
![](pdf_images/Oleg 11_8.pdf-2-0.png)

# Matching

1. Churlish 1. feeling or showing sorrow and remorse for

improper or objectionable behav

In [22]:
query_engine = index.as_query_engine(verbose=True, similarity_top_k=2)
result = query_engine.query('''
                            The following examples from PDFs:
                                Churlish - The new neighbor behaved churlishly at the beginning but, with
                                Buffet (v) - The weaves started buttering the boat of the boat, signaling
that it was time to return to the port.
                                Succulent - The main dish for the day was a succulent New York steak
                                Clandestine - Two parties negotiated peace through a series of
                                
                            Based on provided examples return the list of words and example sentences following the format in the example.
                            Please add 10 more words from PDFs.
                            ''')
print(result)

1. Churlish
The new neighbor behaved churlishly at the beginning but, with time, became friendlier.

2. Buffet (v)
The weaves started buttering the boat of the ship, signaling that it was time to return to the port.

3. Succulent
The main dish for the day was a succulent New York steak.

4. Clandestine
Two parties negotiated peace through a series of clandestine meetings.

5. Bemuse
The presenter entered the wrong room, which bemused the listeners.

6. Buffet (n)
The celebration finished with a big party accompanied by a delicious buffet.

7. Complacent
After many months in the gym, he felt complacent and stopped pushing hard.

8. Churlishness
His churlishness towards others was well-known throughout the neighborhood.

9. Succulently
She cooked the steak succulently, bringing out its true flavor.

10. Clandestine
The group conducted their secret meetings clandestinely, avoiding detection by authorities.

11. Buffet
The hotel offered a buffet service for all guests, with multiple option

In [ ]:
from llama_index.core.evaluation import generate_question_context_pairs
nodes = index.docstore.aget_nodes()
qa_dataset = generate_question_context_pairs(
    nodes, llm=llm, num_questions_per_chunk=2
)